In [52]:
from bs4 import BeautifulSoup
import requests

In [53]:
home_url = 'https://www.simplytarot.com'
base_urls = [
    'https://www.simplytarot.com/tarot-card-meanings/major-arcana-tarot-card-meanings/',
    'https://www.simplytarot.com/tarot-card-meanings/minor-arcana/swords/',
    'https://www.simplytarot.com/tarot-card-meanings/minor-arcana/cups/',
    'https://www.simplytarot.com/tarot-card-meanings/minor-arcana/wands/',
    'https://www.simplytarot.com/tarot-card-meanings/minor-arcana/pentacles/'
]
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}

In [54]:
bodies = {}
for url in base_urls:
    html_body = requests.get(
        url,
        headers=headers).content
    bodies[url] = BeautifulSoup(html_body).find_all('div', class_='cardContainer')

In [56]:
cards = {}
errors = []
for url, body in bodies.items():
    for card in body:
        card_name = card.find('img')['alt'].replace('-tarot-card', '')

        response = requests.get(url + card_name, headers=headers)
        
        if response.status_code > 300:
            errors.append(response)
        else:
            cards[card_name] = response.content

In [66]:
cards_formatted = []
for card, body in cards.items():
    card_bs = BeautifulSoup(body)
    
    content = [c for c in card_bs.find_all('div', class_='card') if c.get_text() != ''][0]
    
    if content != None:
        image_url = 'https://www.simplytarot.com' + card_bs.find('', class_='cardImgLink').img['src']
        parragraphs = content.find_all('p')
    
        cards_formatted.append({
            'name': card,
            'summary': parragraphs[0].get_text(),
            'full_meaning': parragraphs[2].get_text(),
            'image': image_url
        })

In [67]:
cards_formatted

[{'name': 'the-fool',
  'summary': 'Short Meaning: \xa0A new phase in life and an urge to explore new aspects of yourself. Decisions have to be thought through carefully; don’t lead yourself into unsuitable or difficult situations.',
  'full_meaning': 'In-depth Meaning: \xa0Something new is starting for the Subject which could be a new relationship, job or home.\xa0\xa0 This is likely to be an exciting time for them with lots of hope and anticipation for the future.\xa0 The Subject is so caught up in the buzz of it all that they may not be thinking as clearly as they should.',
  'image': 'https://www.simplytarot.com/wp-content/uploads/2015/04/the-fool-tarot-card.png'},
 {'name': 'the-magician',
  'summary': 'Short meaning: \xa0A crossroads. An opportunity for constructive change and for you to establish a new, better reality for yourself. You have everything you need to succeed. Capitalise on your skills and creative talents.',
  'full_meaning': 'In-depth meaning: \xa0When The Magician

In [68]:
import json

with open('cards.json', 'w') as file:
    json.dump(cards_formatted, file)